In [1]:
%%writefile fhn1Ppred.py

from modulus.models.fully_connected import FullyConnectedArch
from modulus.models.fourier_net import FourierNetArch
from modulus.models.siren import SirenArch
from modulus.models.modified_fourier_net import ModifiedFourierNetArch
from modulus.models.dgm import DGMArch

from sympy import Symbol, Eq
from sympy import Symbol, Function, Number
from modulus.eq.pde import PDE
import numpy as np
import matplotlib.pyplot as plt
from scipy import interpolate
import modulus
from modulus.hydra import instantiate_arch, ModulusConfig
from modulus.solver import Solver
from modulus.domain import Domain
from modulus.geometry.primitives_1d import Point1D
from modulus.domain.constraint import (
    PointwiseBoundaryConstraint,
    PointwiseInteriorConstraint,
)
from modulus.domain.validator import PointwiseValidator
from modulus.key import Key
from modulus.node import Node
from modulus.eq.pde import PDE
from modulus.geometry import Parameterization
from sympy import Symbol, Eq, Abs, tanh, Or, And
from modulus.utils.io import (
    csv_to_dict,
    ValidatorPlotter,
    InferencerPlotter,
)
from modulus.solver import SequentialSolver

from modulus.models.deeponet import DeepONetArch
from modulus.domain.constraint.continuous import DeepONetConstraint
from modulus.models.moving_time_window import MovingTimeWindowArch
from modulus.domain.monitor import Monitor
from modulus.domain.constraint import Constraint
from modulus.graph import Graph
from modulus.key import Key
from modulus.constants import TF_SUMMARY
from modulus.distributed import DistributedManager
from modulus.utils.io import dict_to_csv, csv_to_dict
from modulus.domain.inferencer.pointwise import PointwiseInferencer as PointwiseInferencer
from modulus.loss.loss import CausalLossNorm

  
import os
import time
import numpy as np
import torch
from torch.utils.tensorboard import SummaryWriter
from torch.optim import Optimizer
from torch.optim.lr_scheduler import _LRScheduler
from torch.cuda.amp import GradScaler
import torch.nn as nn
import torch.cuda.profiler as profiler
import torch.distributed as dist
from termcolor import colored, cprint
from copy import copy
from operator import add
from omegaconf import DictConfig, OmegaConf
import hydra
import itertools
from collections import Counter
from typing import Dict, List, Optional
import logging
from contextlib import ExitStack
from typing import List, Union, Tuple, Callable


from modulus.domain.constraint import Constraint
from modulus.domain import Domain
from modulus.loss.aggregator import Sum
from modulus.utils.training.stop_criterion import StopCriterion
from modulus.constants import TF_SUMMARY, JIT_PYTORCH_VERSION
from modulus.hydra import (
    instantiate_optim,
    instantiate_sched,
    instantiate_agg,
    add_hydra_run_path,
)
from modulus.distributed.manager import DistributedManager

import contextlib
import io
import sys
import time as TIME
from load import load
global nw
t_max = 10.0
n_w=1
t_w= t_max/n_w    
class SSolver(SequentialSolver):
    def __init__(  self,
        cfg: DictConfig,
        domains: List[Tuple[int, Domain]],
        custom_update_operation: Union[Callable, None] = None,
    ):
        SequentialSolver.__init__(self,cfg,domains,custom_update_operation)
  

    
    @property
    def network_dir(self):
        dir_name ="/home/jovyan/final/1P/outputs/fhn1P/"+ self.domain.name
        if self.domains[self.domain_index][0] > 1:
            dir_name += "_" + str(self.iteration_index).zfill(4)
        return dir_name

    def setnetwork_dir(self,dir):
        self.network_dir=dir
    def eval(
        self,
    ):
        # check the directory exists


        # create global model for restoring and saving

    
        #print(self.domains)
        for domain_index in range(0, len(self.domains)):
                # solve for number of iterations in train_domain
            for iteration_index in range(0, self.domains[domain_index][0]   ):

                    # set internal domain index and iteration index
                    self.domain_index = domain_index
                    self.iteration_index=iteration_index 
                    self.log.info(
                        "Predicting for Domain "
                        + str(self.domain.name)
                        + ", iteration "
                        + str(self.iteration_index)
                    )
                    
                    if not os.path.exists(self.network_dir):
                        print(os.getcwd()+self.network_dir)
                        raise RuntimeError("Network checkpoint is required for eval mode.")
                    self.saveable_models = self.get_saveable_models()

        # set device
                    if self.device is None:
                        self.device = self.manager.device

        # load model
                    self.step = self.load_step()
                    self.step = self.load_model()
                    self.step_str = f"[step: {self.step:10d}]"

                    # make summary writer
                    self.writer = SummaryWriter(
                        log_dir=self.network_dir, purge_step=self.summary_freq + 1
                    )
                    self.summary_histograms = self.cfg["summary_histograms"]

                    if self.manager.cuda:
                        torch.cuda.synchronize(self.device)

                    # write inference / validation datasets to tensorboard and file
                    if self.has_validators:
                        self._record_validators(self.step)
                    if self.has_inferencers:
                        self._record_inferencers(self.step)
                    if self.has_monitors:
                        self._record_monitors(self.step)

def generateExactSolution(t,dt,x0,rate,P,begin,end):
    
    
    n2=int(t/(dt))+2
    n = int((end-begin)/(dt*rate))
    Sol=np.zeros((n,3))
  
    return Sol
def generateValidator(w_i,nodes):

    
    #print(np.shape(X))
    if(n_w==1):
        T=Input
    else:
        T=Input[w_i]-w_i
    print(len(X0))
    print(np.shape(X0))
    print("lenT",len(T))
    t=np.zeros(len(T)*len(X0))
    x=np.zeros(len(T)*len(X0))

    i=0
    k=0
    j=0
    print(T)
    for x0 in X0:
        j=0
        for tt in T:
            t[i]=T[j]
            x[i]=X0[k]
            i=i+1
            j=j+1
        k=k+1
    print(t,x)
    #print(np.shape(ASD))
    invar_numpy = {"t":np.expand_dims( t,-1),"K":np.expand_dims(x,-1)}
    outvar = {"x1","w"}
   
    validator = PointwiseInferencer(
        nodes=nodes, invar=invar_numpy,
        output_names=outvar, batch_size=len(X0)*100,plotter=None
    )
    return validator


def generateDataC(w_i,nodes):

    
    
    T=np.empty([0])
    K=np.empty([0])
    SOLs=np.empty([0])
    SOLw=np.empty([0])
    krange= [(0.01 + 0.06*i*1) for i in range(0,20)]

    deltaT = 0.01
    
    rate=1
    for KR in krange:
        begin=w_i* t_w
        end=begin + t_w
        sol=generateExactSolution(t_max,deltaT,KR,rate,KR,begin,end) ##NAO RODA, RETORNA MATRIZ DE ZEROS
        
        T=np.append(T,np.linspace(0,1,100))
        K = np.append(K,np.full_like (sol.T[2],KR))
        SOLs=np.append(SOLs,sol.T[0])
        SOLw=np.append(SOLw,sol.T[1])
    
    
    t = np.expand_dims(T, axis=-1)


    k = np.expand_dims(K, axis=-1)

    
    Solx = np.expand_dims(SOLs, axis=-1)

    
    Solw = np.expand_dims(SOLw, axis=-1)
   
    
    print(t,"val set de ",begin,"a ", end)
    print(np.shape(t),np.shape(k))
    
    invar_numpy = {"t": t,"K":k}
    outvar_numpy = {
        "x1": Solx,
        "w":Solw
    }
   
    data = DeepONetConstraint.from_numpy(
        nodes=nodes,
        invar=invar_numpy,
        outvar=outvar_numpy,
        batch_size=len(krange)*10,
        lambda_weighting=None
    )
    return data


class SpringMass(PDE):
    name = "SpringMass"

    def __init__(self):

        t = Symbol("t")
       
        input_variables = {"t": t}

        x = Function("x1")(*input_variables)
        w= Function("w")(*input_variables)
        self.equations = {}
        self.equations["ode_x1"] =10*(x*(x-0.4)*(1-x)-w) -x.diff(t)
        self.equations["ode_w"]  =0.5*(x*0.2-0.8*w) -w.diff(t)
        
@modulus.main(config_path="conf", config_name="config")
def run(cfg: ModulusConfig) -> None:
    
    # make list of nodes to unroll graph on
    sm = SpringMass()
    sm.pprint()
       
    
    flow_net = FullyConnectedArch(
            input_keys=[Key("t"), Key("K") ],
            output_keys=[Key("x1"),Key("w")],
            layer_size=36,
            nr_layers=4,
        )

    

    time_window_net = MovingTimeWindowArch(flow_net, t_w)

    nodes = sm.make_nodes() +[time_window_net.make_node(name="network")]


    for node in nodes:
        print(node.__str__())
   
    # add constraints to solver
    # make geometry
    geo = Point1D(0)
    
    t_symbol = Symbol("t")
    x_symbol = Symbol("x1")
    k_symbol= Symbol("K")
    time_range = {t_symbol: (0,t_w )}
    k_range= {k_symbol:(0,1)}
    tr = {t_symbol: (0, t_w)}

    # make domain
        # make initial condition domain
    ic_domain = Domain("initial_conditions")

  
    # initial conditions
    IC = PointwiseBoundaryConstraint(
        nodes=nodes,
        geometry=geo,
        outvar={"x1": k_symbol,"w":0},
        batch_size=2000,
        parameterization={**{t_symbol:0},**k_range},
        lambda_weighting={
            "x1": 100,# + 1000*x_symbol.diff(t_symbol)*x_symbol.diff(t_symbol),
            "w": 100 #+ 1000*x_symbol.diff(t_symbol)*x_symbol.diff(t_symbol)
        },
        
        quasirandom=True,
    )

    ic_domain.add_constraint(IC, name="IC")
    
        # solve over given time period
    interior = PointwiseBoundaryConstraint(
        nodes=nodes,
        geometry=geo,
        outvar={"ode_x1": 0.0,"ode_w":0.0},
        batch_size=1000,
        parameterization={**tr,**k_range},
        #criteria=And(t_symbol > 0, t_symbol < 3),
        lambda_weighting={
            "ode_x1": 100,# + 1000*x_symbol.diff(t_symbol)*x_symbol.diff(t_symbol),
            "ode_w": 100 #+ 1000*x_symbol.diff(t_symbol)*x_symbol.diff(t_symbol)
        },
        quasirandom=True,
    )
    ic_domain.add_constraint(interior, name="interior")

    
    
    
    domains=[]
    for i in range(1,n_w):

        # make moving window domain
        window_domain = Domain("window"+str(i))

        # solve over given time period
        interior1 = PointwiseBoundaryConstraint(
        nodes=nodes,
        geometry=geo,
        outvar={"ode_x1": 0.0,"ode_w":0.0},
        batch_size=2000,
        parameterization={**tr,**k_range},
        #criteria=And(t_symbol > 0, t_symbol < 3),
        lambda_weighting={
            "ode_x1": 100,# + 1000*x_symbol.diff(t_symbol)*x_symbol.diff(t_symbol),
            "ode_w": 100 #+ 1000*x_symbol.diff(t_symbol)*x_symbol.diff(t_symbol)
        },
        quasirandom=True,

        )
        interior2 = PointwiseBoundaryConstraint(
        nodes=nodes,
        geometry=geo,
        outvar={"ode_x1": 0.0,"ode_w":0.0},
        batch_size=1000,
        parameterization={**tr,**{k_symbol:(0.45,0.6)}},
        #criteria=And(t_symbol > 0, t_symbol < 3),
        lambda_weighting={
            "ode_x1": 200,# + 1000*x_symbol.diff(t_symbol)*x_symbol.diff(t_symbol),
            "ode_w": 200 #+ 1000*x_symbol.diff(t_symbol)*x_symbol.diff(t_symbol)
        },
        quasirandom=True,

        )
        
        
        IC = PointwiseBoundaryConstraint(
        nodes=nodes,
        geometry=geo,
        outvar={"x1_prev_step_diff":0,"w_prev_step_diff":0},
        batch_size=500,
        parameterization={**{t_symbol:0},**k_range},
        lambda_weighting={
            "x1_prev_step_diff": 1000,
            "w_prev_step_diff": 1000
        },
        
        quasirandom=True,
    )
        
    
        
        window_domain.add_constraint(IC, name="IC")
        window_domain.add_constraint(interior1, "interior")
        #window_domain.add_constraint(interior2, "interiorTr")

        domains.append(window_domain)
    

    
 
    
    

    
    dom=[]
    dom.append((1,ic_domain))

    for domain in domains:
        dom.append((1,domain))
    print(cfg)
    # make solver
    #slv = Solver(cfg, domain)
    #print(domains)
    i=0
    for a,d in dom:
        print(d)
        print(d.name)
        d.add_inferencer(generateValidator(i,nodes))
        d.add_constraint(generateDataC(i,nodes))
        i=i+1
    
      
    start_time = time.time()
  
    slv = SSolver(
            cfg,
            dom,
            custom_update_operation=time_window_net.move_window,

    )
    slv.eval()
    end_time = time.time()
    elapsed_time = end_time - start_time
    global e
    e=elapsed_time


        
        
        
def join():
    # Define the input arrays
    array1 = [1, 2, 3]
    array2 = ['A', 'B']

    # Get the lengths of the input arrays
    len1 = len(array1)
    len2 = len(array2)

    # Create an empty matrix to hold the combinations
    matrix = np.empty((len1*len2, 2), dtype=object)

    # Fill the matrix with the combinations
    for i in range(len1):
        for j in range(len2):
            matrix[i*len2+j, 0] = array1[i]
            matrix[i*len2+j, 1] = array2[j]

    # Print the resulting matrix
    return matrix


def eval(X,t):

    with contextlib.redirect_stdout(io.StringIO()):
    #if(True):
        global Input
        global X0
        X0=X
        ASD= np.expand_dims(t[0][1:],-1)
        if(1==1):
            Input=ASD    
        else:
            print(np.shape(ASD))
            intervals = np.array([int(i * 1 )for  i in range(0,10+1)])
            print(intervals)
            Input=  [ASD[(ASD >= intervals[i]) & (ASD < intervals[i+1])] for i in range(len(intervals)-1)]
        run()     
        u,i=load(X)

    return u, e




from FHNCUDAlib import FHNCUDA
import numpy as np
from plotter import call
import time as TIME
x0=np.expand_dims(np.array([(0.03 + 0.0001*i*1) for i in range(0,10000)]),-1)
dt,tt=0.01,10



start_time = TIME.time()
u,v ,t=FHNCUDA.run(x0,tt,dt,20)
reftime = TIME.time()- start_time

u_ref=np.array(u).flatten()

print("Shape ref ",np.shape(u_ref))
print("ref cuda time",reftime)




start_time = TIME.time()
u,v ,t=FHNCUDA.run(x0,tt,dt*10,20/10)
cudatime = TIME.time()- start_time
u_num=np.array(u).flatten()

print("Shape cudapred ",np.shape(u_num))
print("cuda time",cudatime)
print("Error Calculation")
e=((u_ref-u_num)**2)**(1/2)
print("mean",np.mean(e))
m=np.max(e)
print("max",m)
i=[a for a in range(len(e)) if e[a]==m]
#print("id of max",i, u_num[i], "  -  ",u_net[i])



start_time = TIME.time()
u,time=eval(x0,t)
nettime = TIME.time()- start_time
u_net=u.flatten()

print("Shape netpred ",np.shape(u_net))
print("net time", nettime)
print("Error Calculation")
e=((u_ref-u_net)**2)**(1/2)
print("mean",np.mean(e))
m=np.max(e)
print("max",m)
i=[a for a in range(len(e)) if e[a]==m]
#print("id of max",i, u_num[i], "  -  ",u_net[i])




#print(u_net)







plt.plot(u_num,"b")
plt.plot(u_net,"r")
plt.savefig("aa.png")





e=lambda x:np.array(x).T

invar={"t":e(t )  ,"K":e(x0)   }
out={"x1":e(u_net),       }
out_t={"x1":e(u_num),         }



Overwriting fhn1Ppred.py


In [2]:
!python fhn1Ppred.py

5
Number of rows in the CSV file:  10000 51
Shape ref  (510000,)
ref cuda time 0.8291070461273193
5
Number of rows in the CSV file:  10000 50
Shape cudapred  (500000,)
cuda time 0.8000023365020752
Error Calculation
Traceback (most recent call last):
  File "/home/jovyan/final/1P/fhn1Ppred.py", line 539, in <module>
    e=((u_ref-u_num)**2)**(1/2)
ValueError: operands could not be broadcast together with shapes (510000,) (500000,) 


In [3]:

%%writefile predComp.py



Overwriting predComp.py


In [4]:
!python predComp.py

In [5]:
    
import numpy as np
import torch as pt
import torch.nn as nn
import torch.nn.functional as f
import torch.optim as optim
import collections as coll
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
from matplotlib import cm


PATH = "outputs/fhn1P/initial_conditions/network.0.pth"
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()


    def forward(self, x):
        return x

    
pt.set_grad_enabled (False) 
numinputs=1
numoutputs=2
class net(nn.Module):
    def __init__(self, numinputs, numoutputs):
        super(net, self).__init__()
        heigth=36
        
        self.layer0 = nn.utils.weight_norm(nn.Linear(numinputs, heigth),name='weight', dim=0).cuda()
        self.layer1 = nn.utils.weight_norm(nn.Linear(heigth, heigth),name='weight', dim=0).cuda()
        self.layer2 = nn.utils.weight_norm(nn.Linear(heigth, heigth),name='weight', dim=0).cuda()
        self.layer3 = nn.utils.weight_norm(nn.Linear(heigth, heigth),name='weight', dim=0).cuda()
        self.layer4 = nn.utils.weight_norm(nn.Linear(heigth, heigth),name='weight', dim=0).cuda()

        self.layer10 = nn.Linear(heigth, numoutputs).cuda()
        
        self.layer0.eval()
        self.layer1.eval()
        self.layer2.eval()
        self.layer3.eval()
        self.layer10.eval()

        
    def forward(self, x):
        x0 = f.silu(self.layer0(x))
        x1 = f.silu(self.layer1(x0))
        x2 = f.silu(self.layer2(x1))
        x3 = f.silu(self.layer3(x2))
        x4 = f.silu(self.layer4(x3))
 
 
        return self.layer10(x3)
    
    def load(self, od):
        for k, v in od.items():
            print(k,"loaded as")
            if(k == '_impl.layers.0.linear.weight'): 
                print('Loading: ',k, np.shape(v), ' -> self.layer0.weight.data')
                self.layer0.weight_v.data = v
                self.layer0.weight_v.requires_grad = False
            if(k == '_impl.layers.0.linear.weight_g'):
                print('Loading: ',k, np.shape(v), ' -> self.layer0.weight_g.data')
                self.layer0.weight_g.data = v
                self.layer0.weight_g.requires_grad = False
            if(k == '_impl.layers.0.linear.bias'):
                print('Loading: ',k, np.shape(v), ' -> self.layer0.bias.data')
                self.layer0.bias.data = v
                self.layer0.bias.requires_grad = False
            if(k == '_impl.layers.1.linear.weight'):
                print('Loading: ',k, np.shape(v), ' -> self.layer1.weight.data')
                self.layer1.weight_v.data = v
                self.layer1.weight_v.requires_grad = False
            if(k == '_impl.layers.1.linear.weight_g'):
                print('Loading: ',k, np.shape(v), ' -> self.layer1.weight_g.data')
                self.layer1.weight_g.data = v
                self.layer1.weight_g.requires_grad = False
            if(k == '_impl.layers.1.linear.bias'):
                print('Loading: ',k, np.shape(v), ' -> self.layer1.bias.data')
                self.layer1.bias.data = v
                self.layer1.weight_g.requires_grad = False
            if(k == '_impl.layers.2.linear.weight'):
                print('Loading: ',k, np.shape(v), ' -> self.layer2.weight.data')
                self.layer2.weight_v.data = v
                self.layer2.weight_v.requires_grad = False
            if(k == '_impl.layers.2.linear.weight_g'):
                print('Loading: ',k, np.shape(v), ' -> self.layer2.weight_g.data')
                self.layer2.weight_g.data = v
                self.layer2.weight_g.requires_grad = False
            if(k == '_impl.layers.2.linear.bias'):
                print('Loading: ',k, np.shape(v), ' -> self.layer2.bias.data')
                self.layer2.bias.data = v
                self.layer2.bias.requires_grad = False
                
                
            if(k == '_impl.layers.3.linear.weight'):
                print('Loading: ',k, np.shape(v), ' -> self.layer3.weight.data')
                self.layer3.weight_v.data = v
                self.layer3.weight_v.requires_grad = False
            if(k == '_impl.layers.3.linear.weight_g'):
                print('Loading: ',k, np.shape(v), ' -> self.layer3.weight_g.data')
                self.layer3.weight_g.data = v
                self.layer3.weight_g.requires_grad = False
            if(k == '_impl.layers.3.linear.bias'):
                print('Loading: ',k, np.shape(v), ' -> self.layer3.bias.data')
                self.layer3.bias.data = v
                self.layer3.bias.requires_grad = False  
                
                
            if(k == '_impl.final_layer.linear.weight'):
                print('Loading: ',k, np.shape(v), ' -> self.layer10.weight.data')
                self.layer10.weight.data = v
                self.layer10.weight.requires_grad = False
            if(k == '_impl.final_layer.linear.bias'):
                print('Loading: ',k, np.shape(v), ' -> self.layer10.bias.data')
                self.layer10.bias.data = v
                self.layer10.bias.requires_grad = False
            print("---")

    def dumpLayerWeightSizes(self):
        print(self.layer0.weight.size())
        print(self.layer1.weight.size())
        print(self.layer2.weight.size())
        print(self.layer3.weight.size())
        print(self.layer4.weight.size())


    def dumpLayerWeights(self):
        print(self.layer0.weight)
        print(self.layer1.weight)
        print(self.layer2.weight)
        print(self.layer3.weight)
        print(self.layer4.weight)

    def dumpLayerWeightGSizes(self):
        print(self.layer0.weight_g.size())
        print(self.layer1.weight_g.size())
        print(self.layer2.weight_g.size())
        print(self.layer3.weight_g.size())
        print(self.layer4.weight_g.size())

    def dumpLayerWeightGs(self):
        print(self.layer0.weight_g)
        print(self.layer1.weight_g)
        print(self.layer2.weight_g)
        print(self.layer3.weight_g)
        print(self.layer4.weight_g)

    def dumpLayerBiasSizes(self):
        print(self.layer0.bias.size())
        print(self.layer1.bias.size())
        print(self.layer2.bias.size())
        print(self.layer3.bias.size())
        print(self.layer4.bias.size())

    def dumpLayerBiases(self):
        print(self.layer0.bias)
        print(self.layer1.bias)
        print(self.layer2.bias)
        print(self.layer3.bias)
        print(self.layer4.bias)
    
    
    
    

net =net(2,2)

od=pt.load(PATH)
net.load(od)

model=net
import itertools

def Modelrun(x=np.linspace(0, 1, num=10),t=np.linspace(0,10,num=10)):
   
    
    X=np.zeros((2,len(t)*len(x)))
    #print(itertools.product(x,t))
    i=0
    for a,b in itertools.product(x,t):
        X[:,i]=(b,a)
        i=i+1
    #print(np.shape(X))
    my2dspace = pt.tensor(X.T, requires_grad=False)
    model.eval()

    myOutput = model(my2dspace.float().cuda())
    myCPUOutput = myOutput.cpu()


    uu = myCPUOutput.numpy()

    #print('uu: ', uu.T[0])

    myCPUOutput.squeeze().detach().numpy()
    return uu


/opt/saturncloud/envs/saturn/lib/python3.9/site-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


_impl.layers.0.linear.weight loaded as
Loading:  _impl.layers.0.linear.weight torch.Size([36, 2])  -> self.layer0.weight.data
---
_impl.layers.0.linear.weight_g loaded as
Loading:  _impl.layers.0.linear.weight_g torch.Size([36, 1])  -> self.layer0.weight_g.data
---
_impl.layers.0.linear.bias loaded as
Loading:  _impl.layers.0.linear.bias torch.Size([36])  -> self.layer0.bias.data
---
_impl.layers.1.linear.weight loaded as
Loading:  _impl.layers.1.linear.weight torch.Size([36, 36])  -> self.layer1.weight.data
---
_impl.layers.1.linear.weight_g loaded as
Loading:  _impl.layers.1.linear.weight_g torch.Size([36, 1])  -> self.layer1.weight_g.data
---
_impl.layers.1.linear.bias loaded as
Loading:  _impl.layers.1.linear.bias torch.Size([36])  -> self.layer1.bias.data
---
_impl.layers.2.linear.weight loaded as
Loading:  _impl.layers.2.linear.weight torch.Size([36, 36])  -> self.layer2.weight.data
---
_impl.layers.2.linear.weight_g loaded as
Loading:  _impl.layers.2.linear.weight_g torch.Size([

In [6]:
from FHNCUDAlib import FHNCUDA
x0=np.expand_dims(np.linspace(0,1,10),-1)
dt,tt=0.01,10
t=np.expand_dims(np.linspace(0,tt,int(tt/dt)),-1)



u,v,t=FHNCUDA.run(x0,tt,dt,1)
print(np.shape(u),np.shape((np.array(v)).flatten()))
print(np.shape(t))

2
Number of rows in the CSV file:  10 1000
(10, 1000) (10000,)
(1, 1001)


In [7]:
import os
os.chdir("1P")

FileNotFoundError: [Errno 2] No such file or directory: '1P'

In [ ]:
import numpy as np
from load import load
X=np.linspace(0,2,10)
a=load(X)

In [ ]:

from FHNCUDAlib import FHNCUDA
import numpy as np
from plotter import call
import time as TIME
x0=np.expand_dims(np.array([(0.03 + 0.00005*i*1) for i in range(0,20000)]),-1)
dt,tt=0.0001,10



start_time = TIME.time()
u,v ,t=FHNCUDA.run(x0,tt,dt,200)
reftime = TIME.time()- start_time

u_ref=np.array(u).flatten()

print("Solving for ",len(x0),"cells in " ,int(np.shape(u_ref)[0]/len(x0)),"timepoints" )
print("Shape ref ",np.shape(u_ref))
print("ref cuda time",reftime)




start_time = TIME.time()
u,v ,t=FHNCUDA.run(x0,tt,dt*100,200/100)
cudatime = TIME.time()- start_time
u_num=np.array(u).flatten()
#print(np.unique(t))

print("Shape cudapred ",np.shape(u_num))
print("cuda time",cudatime)
print("Error Calculation")
e=((u_ref-u_num)**2)**(1/2)
print("mean",np.mean(e))
m=np.max(e)
print("max",m)
i=[a for a in range(len(e)) if e[a]==m]
#print("id of max",i, u_num[i], "  -  ",u_net[i])

x0 = [item for sublist in x0 for item in sublist]
t = [item for sublist in t for item in sublist]
#print(x0)
#print(np.shape(x0),np.shape(t))

start_time = TIME.time()
u=Modelrun(x0,t[1:])
nettime = TIME.time()- start_time

u_net=u.T[0].flatten()

print("Shape netpred ",np.shape(u_net))
print("net time", nettime)
print("Error Calculation")
e=((u_ref-u_net)**2)**(1/2)
print("mean",np.mean(e))
m=np.max(e)
print("max",m)
i=[a for a in range(len(e)) if e[a]==m]
#print("id of max",i, u_num[i], "  -  ",u_net[i])



#print(u_net)







plt.plot(u_num,"b")
plt.plot(u_net,"r")
plt.savefig("aa.png")





e=lambda x:np.array(x).T

invar={"t":e(t )  ,"K":e(x0)   }
out={"x1":e(u_net),       }
out_t={"x1":e(u_num),         }
